# CHAPTER 1. Basic Operations on Delta Lakes

* 목차
  - [1. What is Delta Lake?](#1.-What-is-Delta-Lake?)
  - [2. How to start using Delta Lake](#2.-How-to-start-using-Delta-Lake)
  - [3. Basic operations](#3.-Basic-operations)
  - [4. Unpacking the Transaction Log](#4.-Unpacking-the-Transaction-Log)
  - [5. Table Utilities](#5.-Table-Utilities)
  - [6. Summary](#6.-Summary)

## Delta Lake quickstart

> 델타 레이크는 Apache Spark 에 ACID 트랜젝션을 지원해줄 수 있는 오픈소스 스토리지 레이어입니다.


```Dockerfile
# Install Delta Lake
RUN pip install jip
RUN jip install "io.delta:delta-core_2.12:0.7.0"
```

* 델타 테이블
  - [Delta Table Documentation](https://docs.delta.io/latest/api/python/index.html)
  - [spark.databricks.delta Configuration](https://books.japila.pl/delta-lake-internals/DeltaSQLConf/)

* 레퍼런스
  - [Delta IO on Github](https://github.com/delta-io/delta)
  - [Delta on Maven](https://mvnrepository.com/artifact/io.delta/delta-core)
  - [Docker Stack Recipe](https://jupyter-docker-stacks.readthedocs.io/it/latest/using/recipes.html)
  - [Data Time Travel by Delta Time Machine](https://databricks.com/session_eu20/data-time-travel-by-delta-time-machine-2) - how each concurrent writes are handled
  - [Data Engineer Training](https://github.com/psyoblade/data-engineer-training)
  - [SparkSQL Magic](https://github.com/cryeo/sparksql-magic)
  - [Introducing Delta Time Travel for Large Scale Data Lakes](https://databricks.com/blog/2019/02/04/introducing-delta-time-travel-for-large-scale-data-lakes.html)
  - [Query an older snapshot of a table (time travel)](https://docs.databricks.com/delta/delta-batch.html?_ga=2.97316378.466340956.1612291545-1203283123.1598416965#query-an-older-snapshot-of-a-table-time-travel)
  - [Unpacking the Transaction Log I](https://databricks.com/discover/diving-into-delta-lake-talks/unpacking-transaction-log)
  - [Unpacking the Transaction Log I](https://databricks.com/session_eu20/diving-into-delta-lake-unpacking-the-transaction-log)
  - [Table Batch Read & Write](https://docs.databricks.com/delta/delta-batch.html)

* 플러그인 및 도구
  - [Spark SQL on Notebook](https://github.com/jupyter-incubator/sparkmagic)
  - [IPython SQL on Notebook](https://github.com/catherinedevlin/ipython-sql)


* 질문 사항
  - 왜 설치된 상태로 수행할 수 없고, 매번 delta-core 를 다운로드 받아야 하는가?
  - 온 프레미스 환경에서 delta lake 사용을 위한 설치환경은 어떻게 구성해야 하는가? (ex_ JupyterHub + Hadoop3 RBF + normal YARN)
  - addpyfile 통해서 추가했을 때에 import 는 되었지만, 실제 method 호출 시에 실패했는데 어떤 이유인가?
  - 결국 체크포인트까지 만들어내면 (트랜젝션 로그의 과거 파일을 삭제하지 않으면) 스몰파일 문제는 계속 심화되는 것 아닌가? 언제 지우는가?
  - 도대체 프로토콜이 언제 왜 사용되는가? 트랜잭션 수행의 순서를 결정짓는 serializability 를 말합니다.
  - 분산환경에서 저장된 델타테이블은 하이브와 같은 메타스토어 그리고 중앙집중식 락 관리도 안 하는 것 처럼 보이는데 왜 가능한가? 그 역할은 누가 어떻게 하는가?
  - 노트북 환경에서 vacuum 실행 시에 상당히 오랜 시간이 걸리는데 왜 그런가?

In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from IPython.display import display, display_pretty, clear_output, JSON

# 노트북에서 델타 레이크를 사용하기 위해 설정을 합니다
spark = (
    SparkSession
    .builder
    .config("spark.sql.session.timeZone", "Asia/Seoul")
    .config("spark.jars.packages", "io.delta:delta-core_2.12:0.7.0")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .getOrCreate()
)


# 노트북에서 테이블 형태로 데이터 프레임 출력을 위한 설정을 합니다
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # display enabled
spark.conf.set("spark.sql.repl.eagerEval.truncate", 100) # display output columns size

# 공통 데이터 위치
home_jovyan = "/home/jovyan"
work_data = f"{home_jovyan}/work/data"
work_dir=!pwd
work_dir = work_dir[0]

In [20]:
from delta.tables import *
spark

## 1. What is Delta Lake?

[맨 위로](#CHAPTER-1.-Basic-Operations-on-Delta-Lakes)

## 2. How to start using Delta Lake

### 2-1. Using Delta Lake via local Spark shells

### 2-2. Leveraging GitHub or Maven

### 2-3. Using Databricks Community Edition

In [3]:
# 임시 경로를 삭제합니다
!pwd
!rm -rf /home/jovyan/work/tmp
!rm -rf /home/jovyan/work/spark-warehouse

/home/jovyan/work/lgde-spark-delta


[맨 위로](#CHAPTER-1.-Basic-Operations-on-Delta-Lakes)

## 3. Basic operations

> 델타 테이블은 Spark, Hive, Presto (or Trino), Ballista (Rust) and AWS Athena, Azure Synapse, BigQuery, and Dremio 등 다양한 오픈소스 및 상용 솔루션에서 사용되어질 수 있도록 설계 되었으며, SQL, Scala 및 Python 등을 통해 수행될 수 있는 APIs 가 존재합니다.

### 3-1. Creating your first Delta table

![write delta](images/figure.1-2.png)

#### Writing your Delta table

In [4]:
rows = [
    Row("정휘센", "안녕하세요 정휘센 입니다", 300),
    Row("김싸이언", "안녕하세요 김싸이언 입니다", 200),
    Row("유코드제로", "안녕하세요 유코드제로 입니다", 100)
]
schema = "`author` string, `title` string, `pages` int"
print(schema)
df = spark.createDataFrame(rows, schema)
df.printSchema()
display(df)

`author` string, `title` string, `pages` int
root
 |-- author: string (nullable = true)
 |-- title: string (nullable = true)
 |-- pages: integer (nullable = true)



author,title,pages
정휘센,안녕하세요 정휘센 입니다,300
김싸이언,안녕하세요 김싸이언 입니다,200
유코드제로,안녕하세요 유코드제로 입니다,100


In [5]:
df.write.format("delta").mode("overwrite").save("tmp/say-hello")

In [6]:
appended = [
    Row("박수혁", "처음 뵙겠습니다", 400),
    Row("김영미", "하지메마시떼", 400)
]
df2 = spark.createDataFrame(appended, schema)
df2.write.format("delta").mode("append").save("tmp/say-hello")

#### Reading your Delta table

In [7]:
df3 = spark.read.format("delta").load("tmp/say-hello")
df3.printSchema()
display(df3)

root
 |-- author: string (nullable = true)
 |-- title: string (nullable = true)
 |-- pages: integer (nullable = true)



author,title,pages
유코드제로,안녕하세요 유코드제로 입니다,100
김싸이언,안녕하세요 김싸이언 입니다,200
정휘센,안녕하세요 정휘센 입니다,300
박수혁,처음 뵙겠습니다,400
김영미,하지메마시떼,400


In [21]:
# 스파크 SQL 상에서는 상대경로를 지정하면 동작하지 않고, 절대경로를 지정해야만 합니다
spark.sql(f"select * from delta.`{work_dir}/tmp/say-hello`")

author,title,pages
유코드제로,안녕하세요 유코드제로 입니다,100
김싸이언,안녕하세요 김싸이언 입니다,200
정휘센,안녕하세요 정휘센 입니다,300
박수혁,처음 뵙겠습니다,400
김영미,하지메마시떼,400


#### Reading your metastore defined Delta table

> 메타데이터 접근을 위해서는 `saveAsTable` 혹은 `CREATE TABLE` 구문을 활용해야만 합니다.

In [22]:
df3.write.format("delta").saveAsTable("hello")
spark.sql("show tables")

database,tableName,isTemporary
default,hello,false


In [23]:
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS say (
        author string,
        title string,
        pages integer
    )
    USING DELTA
    LOCATION "{work_dir}/tmp/say"
""")
spark.sql("show tables")

database,tableName,isTemporary
default,hello,false
default,say,false


> 대용량 테이블 저장 시에는 파티션 설계를 고려하면 좋습니다.

In [25]:
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS say_hello (
        author string,
        title string,
        pages integer
    )
    USING DELTA
    PARTITIONED BY (pages)
    LOCATION "{work_dir}/tmp/say_hello"
""")
spark.sql("show tables")

database,tableName,isTemporary
default,hello,false
default,say,false
default,say_hello,false


#### metastore ?
> 테이블을 정의하기 위한 정보를 저장해 두는 저장소 (ex_ hive metastore)를 말하며, data location, storage format, table schema and properties 등의 정보가 저장됩니다.

[맨 위로](#CHAPTER-1.-Basic-Operations-on-Delta-Lakes)

## 4. Unpacking the Transaction Log

> 델타 레이크의 가장 큰 혜택이 ACID 트랜잭션을 지원하는 것이며 내부적으로 어떻게 동작하는 지 이해할 필요가 있습니다.

* Parquet Table vs. Delta Table
  - Delta 테이블의 경우 `_delta_log` 하는 Delta transaction log 경로가 존재하며, 이를 통해 ACID 트랜잭션 뿐만 아니라 스케일러블한 메타데이터 처리 및 타임 트래블링이 가능해 집니다.

In [28]:
!ls "tmp/say-hello"

_delta_log
part-00000-035b3a33-0a65-4edc-ac6f-fdaf165d0d50-c000.snappy.parquet
part-00000-f05b7216-70cb-45c2-bd80-b2ea8c63397b-c000.snappy.parquet
part-00001-9cd90836-3994-4156-a53d-a37643906aec-c000.snappy.parquet
part-00001-e7cce94f-bc22-4cfb-83f6-b41dc2f5a122-c000.snappy.parquet
part-00002-0492c0ec-c744-4731-9473-c7cd728a8a51-c000.snappy.parquet
part-00002-66a1573a-3a10-4cf6-a643-dd7056209ec2-c000.snappy.parquet


In [30]:
deltaSayHello = spark.read.format("delta").load("tmp/say-hello")
deltaSayHello.write.mode("overwrite").parquet("tmp/say-hello-parquet")

In [31]:
!ls "tmp/say-hello-parquet"

part-00000-bb19ac2a-dd71-40c9-b7c7-60e1d90fbb0b-c000.snappy.parquet
part-00001-bb19ac2a-dd71-40c9-b7c7-60e1d90fbb0b-c000.snappy.parquet
part-00002-bb19ac2a-dd71-40c9-b7c7-60e1d90fbb0b-c000.snappy.parquet
_SUCCESS


### 4-1. What Is the Delta Lake Transaction Log?

> Delta Lake transaction log (혹은 Delta Log) 는 델타 레이크 테이블에서 발생하는 모든 변경사항을 저장하고 있는 순차적인 레코드 입니다.

#### Single Source of Truth

> 항상 이용자에게 올바른 뷰를 보여주기 위해, 사용자가 수행하는 모든 변경사항을 트래킹하는 센트럴 레포지토리는 하나의 소스만을 바라보고 있습니다. 이는 소스코드 관리도구 깃의 .git 디렉토리와 유사하게 동작합니다.

* 작업 처리과정에서 장애 혹은 애플리케이션 오류 등의 이유로 완전히 정리되지 않은 부분 파일(partial files)들이 존재할 수 있으나, 후속 처리 쿼리나 데이터 애플리케이션은 이러한 파일들을 구분할 수 있어야합니다. 

![partial file](images/figure.1-4.png)

* Job1 은 3~4.parquet 파일을 생성하는 작업 과정에서 실패로 인해, 3.parquet 만 생성되었고
* Job2 는 동일한 애플리케이션 재실행을 통해 3~4.parquet 이 제대로 생성 되었다
* 파일 목록만으로는 최종 데이터를 파악하기 어렵지만, 트랜잭션 로그를 활용하면 정상적인 파일을 확인할 수 있습니다.

![transaction log](images/figure.1-5.png)

* t1 시점에서는 작업이 실패했기 때문에 트랜젝션 로그에 저장되지 않았고, 데이터 뷰는 t0 시점과 동일하게 2개가 조회 됩니다.
* t2 시점에서는 작업이 성공했기 때문에 트랜젝션 로그에 2개의 파일이 노출되어 모든 데이터가 조회됩니다.

#### The Implementation of Atomicity on Delta Lake

> 델타 레이크가 원자성(atomicity)을 보장할 수 있는 매커니즘의 핵심은 바로 "트랜잭션 로그"입니다.

### 4-2. How Does the Transaction Log Work?

#### Breaking Down Transactions Into Atomic Commits

> '델타 레이크'는 모든 연산(CRUD)을 아래에 명시된 discrete 한 단계로 분해합니다.

##### Update metadata : 테이블의 이름, 스키마 혹은 파티셔닝의 변경 뿐만 아니라 모든 테이블 메타데이터를 변경합니다
##### Add file : 트랜젝션 로그에 데이터 파일을 추가합니다
##### Remove file : 트랜잭션 로그로부터 파일을 제거합니다
##### Set transaction : 구조화된 스트리밍 작업이 지정된 ID로 마이크로 배치를 커밋했음을 기록합니다
##### Change protocol : Delta Lake 트랜잭션 로그를 최신 소프트웨어 프로토콜로 전환하여 새로운 기능을 활성화합니다
##### Commit info : 커밋에 대한 정보, 작업이 수행된 위치 및 시간을 포함합니다

> 이러한 액션들은 트랜잭션 로그에 순차적으로 저장되며, 커밋이라고 불리는 원자적인 단위이기도 합니다.

<br>

#### Delta Transaction Log Protocol

> 이 장에서는 어떻게 Delta tarnsaction log 가 ACID 한 속성을 분산 파일 시스템에 저장된 대용량 콜렉션에 적용하는지 기술합니다. 프로토콜은 아래의 목적에 부합하도록 설계되었습니다.

##### Serializable ACID Writes : ACID semantic 을 보장하면서 동시에 다수의 저장이 가능합니다
##### Snapshot Isolation for Reads : reader 는 다수의 writer 가 동시에 저장하는 중에도 consistent 한 Delta table 의 snapshot 을 읽을 수 있습니다
##### Scalability to billions of partitions or files : Delta table 에 대한 질의는 하나의 장비 혹은 병렬로 수행될 수 있습니다
##### Self-describing : 모든 Delta table 의 메타데이터는 데이터와 함께 저장됩니다. 이러한 설계가 분리된 metastore 유지하는 부담을 제거할 수 있으며, 파일시스템 도구를 통한 복사만으로 정적인 테이블 복사가 가능하게 됩니다.
##### Support for incremental processing : reader 는 Delta log 를 tail 함으로써 주어진 시간 내에 데이터가 추가되었는지를 인지할 수 있고 효과적인 스트리밍 처리가 가능합니다.

<br>

#### Logstore

> LogStore 는 Delta transaction log 를 읽고 쓰는데에 필요한 파일시스템을 위한 인터페이스입니다. 대부분의 파일 스토리지 시스템은 원자성을 보장하지 않기 때문에 LogStore API 를 통해 서 이용할 수 있습니다.

##### 어떤 파일도 전체가 보이거나 아예 보이지 않도록 하며, partial files 를 생성해내지 않습니다
##### 단 하나의 writer 만이 최종 경로에 파일을 생성할 수 있기 때문에, 수 많은 writers 들이 그 자신의 파일들을 병렬로 쓰는 것은 가능합니다. 
##### Logstore 는 ACID consistent 파일 목록을 제공합니다

<br>

#### The Delta Lake Transaction Log at the File Level

> Delta table 이 생성될 때, `_delta_log` 라는 하위 디렉토리에 트랜잭션 로그가 자동적으로 생성됩니다. 테이블에 대한 변경이 발생할 때에 순차적으로 변경사항에 대하여 트랜잭션 로그에 원자적 커밋들이 저장됩니다. 매 커밋들은 000000.json 파일로 시작하는 JSON 파일 형태로 저장되며, 이어지는 변경 사항들은 숫자가 늘어나면서 000001.json 과 같이 저장됩니다. 이 숫자가 테이블의 `새로운 버전`을 나타냅니다.

![delta on disk](images/figure.1-6.png)

##### Implementing Atomicity : 하나의 온전한 트랜잭션은 하나의 json 파일로 구성되며, 이러한 동작 하나 하나가 커밋의 단위로 관리됩니다

![automicity in delta](images/figure.1-7.png)

> 그림에서와 같이 1.parquet, 2.parquet 파일은 Delta Lake 테이블의 일부가 더 이상 아니지만, 트랜잭션 로그에는 여전히 저장되어지는데, 어떤 트랜잭션도 취소 혹은 다시 되돌릴 수 있기 때문입니다. 이러한 이유로 Delta Lake 는 원자적 커밋을 통해 'time travel' 하거나 'audit' 이 가능하게 됩니다.

In [34]:
# 첫 번째 version 의 트랜잭션 로그를 읽어봅시다 - /work/jovyan/tmp/delta-table/_delta_log/00000000000000000000.json
_delta_log = f"{work_dir}/tmp/say-hello/_delta_log/00000000000000000000.json"
log_v1 = spark.read.json(_delta_log)
log_v1.printSchema()
display(log_v1)

root
 |-- add: struct (nullable = true)
 |    |-- dataChange: boolean (nullable = true)
 |    |-- modificationTime: long (nullable = true)
 |    |-- path: string (nullable = true)
 |    |-- size: long (nullable = true)
 |-- commitInfo: struct (nullable = true)
 |    |-- isBlindAppend: boolean (nullable = true)
 |    |-- operation: string (nullable = true)
 |    |-- operationMetrics: struct (nullable = true)
 |    |    |-- numFiles: string (nullable = true)
 |    |    |-- numOutputBytes: string (nullable = true)
 |    |    |-- numOutputRows: string (nullable = true)
 |    |-- operationParameters: struct (nullable = true)
 |    |    |-- mode: string (nullable = true)
 |    |    |-- partitionBy: string (nullable = true)
 |    |-- timestamp: long (nullable = true)
 |-- metaData: struct (nullable = true)
 |    |-- createdTime: long (nullable = true)
 |    |-- format: struct (nullable = true)
 |    |    |-- provider: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- p

add,commitInfo,metaData,protocol
null,"[false, WRITE, [3, 3729, 3], [Overwrite, []], 1628951091358]",null,null
null,null,null,"[1, 2]"
null,null,"[1628951089003, [parquet], eaf1458e-43a5-4d6b-a738-089d6316ddd1, [], {""type"":""struct"",""fields"":[{...",null
"[true, 1628951089811, part-00000-f05b7216-70cb-45c2-bd80-b2ea8c63397b-c000.snappy.parquet, 1189]",null,null,null
"[true, 1628951089812, part-00001-e7cce94f-bc22-4cfb-83f6-b41dc2f5a122-c000.snappy.parquet, 1243]",null,null,null
"[true, 1628951089812, part-00002-0492c0ec-c744-4731-9473-c7cd728a8a51-c000.snappy.parquet, 1297]",null,null,null


> 트랜잭션 정보는 commit, add 그리고 CRC pieces 에 대한 정보들을 담고 있습니다

##### Commit Information : Delta transaction log 는 메타데이터를 커밋합니다

![metadata](images/delta.commitInfo.png)

In [35]:
commitInfo = log_v1.select("commitInfo").where("commitInfo is not null")
c = commitInfo.withColumnRenamed("commitInfo", "c")
c.select("c.*").show(truncate=False)

+-------------+---------+----------------+-------------------+-------------+
|isBlindAppend|operation|operationMetrics|operationParameters|timestamp    |
+-------------+---------+----------------+-------------------+-------------+
|false        |WRITE    |[3, 3729, 3]    |[Overwrite, []]    |1628951091358|
+-------------+---------+----------------+-------------------+-------------+



##### Add Information : Delta transaction log 가 `add` 한 metadata 는 아래와 같이 확인할 수 있습니다.

![metadata](images/delta.add.png)

> 스파크의 경우 경로가 지정된 경우에는 해당 디렉토리에 포함된 모든 파일의 목록을 읽어오는데(listFrom) 특히 클라우드 스토리지의 경우 매우 비효율적이나, Delta Lake 의 경우 transaction log 통한 파일목록을 가져올 수 있으며, 지정된 version 에 해당하는 파일목록만 사용하기 때문에 인터넷 환경 혹은 분산환경의 저장소에서 페타바이트 수준의 데이터 처리 성능을 높일 수 있습니다


In [36]:
add = log_v1.select("add").where("add is not null")
add.printSchema()
a = add.withColumnRenamed("add", "a")
a.select("a.*").show(truncate=False)

root
 |-- add: struct (nullable = true)
 |    |-- dataChange: boolean (nullable = true)
 |    |-- modificationTime: long (nullable = true)
 |    |-- path: string (nullable = true)
 |    |-- size: long (nullable = true)

+----------+----------------+-------------------------------------------------------------------+----+
|dataChange|modificationTime|path                                                               |size|
+----------+----------------+-------------------------------------------------------------------+----+
|true      |1628951089811   |part-00000-f05b7216-70cb-45c2-bd80-b2ea8c63397b-c000.snappy.parquet|1189|
|true      |1628951089812   |part-00001-e7cce94f-bc22-4cfb-83f6-b41dc2f5a122-c000.snappy.parquet|1243|
|true      |1628951089812   |part-00002-0492c0ec-c744-4731-9473-c7cd728a8a51-c000.snappy.parquet|1297|
+----------+----------------+-------------------------------------------------------------------+----+



##### CRC file : 

In [37]:
!ls tmp/say-hello/_delta_log

00000000000000000000.json  00000000000000000001.json


In [38]:
from pyspark.conf import SparkConf
conf = SparkConf()
conf.getAll()

[('spark.sql.session.timeZone', 'Asia/Seoul'),
 ('spark.jars',
  'file:///root/.ivy2/jars/io.delta_delta-core_2.12-0.7.0.jar,file:///root/.ivy2/jars/org.antlr_antlr4-4.7.jar,file:///root/.ivy2/jars/org.antlr_antlr4-runtime-4.7.jar,file:///root/.ivy2/jars/org.antlr_antlr-runtime-3.5.2.jar,file:///root/.ivy2/jars/org.antlr_ST4-4.0.8.jar,file:///root/.ivy2/jars/org.abego.treelayout_org.abego.treelayout.core-1.0.3.jar,file:///root/.ivy2/jars/org.glassfish_javax.json-1.0.4.jar,file:///root/.ivy2/jars/com.ibm.icu_icu4j-58.2.jar'),
 ('spark.sql.extensions', 'io.delta.sql.DeltaSparkSessionExtension'),
 ('spark.repl.local.jars',
  'file:///root/.ivy2/jars/io.delta_delta-core_2.12-0.7.0.jar,file:///root/.ivy2/jars/org.antlr_antlr4-4.7.jar,file:///root/.ivy2/jars/org.antlr_antlr4-runtime-4.7.jar,file:///root/.ivy2/jars/org.antlr_antlr-runtime-3.5.2.jar,file:///root/.ivy2/jars/org.antlr_ST4-4.0.8.jar,file:///root/.ivy2/jars/org.abego.treelayout_org.abego.treelayout.core-1.0.3.jar,file:///root/.ivy

##### Quickly Recomputing State With Checkpoint Files

> transaction log 생성 과정에서 너무 많은 로그 파일이 생성되면 다시 hadoop small file 문제가 생길 수 있기 때문에, Delta Lake 는 매 10번째 커밋 마다 Parquet format 으로 구성된 checkpoint file 을 생성하는데, 이 체크포인트 파일은 현재 시점까지의 모든 상태를 저장합니다. 즉, 스파크 입장에서는 특정 시점까지의 많고 작은 JSON 파일들을 읽기 보다 효과적인 '지름길'을 가지고 있다고 보면 됩니다.

![commit](images/figure.1-9.png)

> 스파크는 아래와 같이 1개의 json 파일이 있다면 모두 읽어서 결과(3개의 커밋 결과)를 메모리에 캐시(cache_v2)하고, 추가적인 커밋이 없는 한 캐시를 활용합니다.

![cache](images/figure.1-10.png)

> 위의 그림에서 추가적인 5개의 커밋이 발생했으므로, 처음(version 0)부터 모든 파일을 읽어서 캐시(cache_v7)합니다.

![cache-2](images/figure.1-11.png)

> 이후 추가적인 5개의 커밋이 발생하였고, Delta Lake 는 10번째의 커밋이 발생한 이후에 11번째 커밋에서 checkpoint file (0000010.checkpoint.parquet) 파일을 생성합니다. Delta Lake 는 지연된 트랜잭션들을 피하기 위해서, 처음(version 0)부터 현재 시점까지의 정보를 캐시(cache_v12) 합니다.

![cache-3](images/figure.1-12.png)

In [39]:
more_appended = [
    Row("정휘센", "첫 출근입니다", 300),
    Row("김싸이언", "처음 뵙겠습니다", 200),
    Row("유코드제로", "오늘 두 번째 출근이네요", 100)
]
schema = "`author` string, `title` string, `pages` int"
df4 = spark.createDataFrame(more_appended, schema)
df4.write.format("delta").mode("append").save("tmp/say-hello")

In [40]:
# 0 ~ 9 까지 json 파일 생성 이후에 약간의 지연 이후 10 넘버의 checkpoint 생성 이후에 다시 12번까지 생성
iter_schema = "`author` string, `title` string, `pages` int"
for num in range(0, 1000, 100):
    iter_row = [Row("오토봇", "인사-{}".format(num), num)]
    iter_df = spark.createDataFrame(iter_row, iter_schema)
    iter_df.write.format("delta").mode("append").save("tmp/say-hello")

In [41]:
chkpt0 = spark.read.parquet("tmp/say-hello/_delta_log/00000000000000000010.checkpoint.parquet")
chkpt0.printSchema()
display(chkpt0)

root
 |-- txn: struct (nullable = true)
 |    |-- appId: string (nullable = true)
 |    |-- version: long (nullable = true)
 |    |-- lastUpdated: long (nullable = true)
 |-- add: struct (nullable = true)
 |    |-- path: string (nullable = true)
 |    |-- partitionValues: map (nullable = true)
 |    |    |-- key: string
 |    |    |-- value: string (valueContainsNull = true)
 |    |-- size: long (nullable = true)
 |    |-- modificationTime: long (nullable = true)
 |    |-- dataChange: boolean (nullable = true)
 |    |-- stats: string (nullable = true)
 |    |-- tags: map (nullable = true)
 |    |    |-- key: string
 |    |    |-- value: string (valueContainsNull = true)
 |-- remove: struct (nullable = true)
 |    |-- path: string (nullable = true)
 |    |-- deletionTimestamp: long (nullable = true)
 |    |-- dataChange: boolean (nullable = true)
 |-- metaData: struct (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- description:

txn,add,remove,metaData,protocol,commitInfo
null,"[part-00000-f05b7216-70cb-45c2-bd80-b2ea8c63397b-c000.snappy.parquet, [], 1189, 1628951089811, fa...",null,null,null,null
null,"[part-00000-2036c214-e2fd-4061-9d44-eb1046e8b6ce-c000.snappy.parquet, [], 1045, 1628951880429, fa...",null,null,null,null
null,"[part-00001-e7cce94f-bc22-4cfb-83f6-b41dc2f5a122-c000.snappy.parquet, [], 1243, 1628951089812, fa...",null,null,null,null
null,"[part-00002-9134f9ef-6a29-4431-991b-512a3c69fc90-c000.snappy.parquet, [], 964, 1628951883436, fal...",null,null,null,null
null,"[part-00000-b6e0e0ae-e9d6-4bec-a3f7-59ac7c7a5a4e-c000.snappy.parquet, [], 468, 1628951884816, fal...",null,null,null,null
null,"[part-00002-f58e284d-ab69-4374-ab09-6b78682bd92a-c000.snappy.parquet, [], 964, 1628951887619, fal...",null,null,null,null
null,"[part-00000-035b3a33-0a65-4edc-ac6f-fdaf165d0d50-c000.snappy.parquet, [], 468, 1628951095838, fal...",null,null,null,null
null,"[part-00000-2cf95065-04dc-415e-8c54-b7dba86b6fb6-c000.snappy.parquet, [], 468, 1628951891370, fal...",null,null,null,null
null,"[part-00002-49bf0928-b09e-4c32-b519-f7d0a4c1be4a-c000.snappy.parquet, [], 1225, 1628951880420, fa...",null,null,null,null
null,"[part-00000-9a8c9c48-ab88-44c1-bf25-cf73853cd396-c000.snappy.parquet, [], 468, 1628951882009, fal...",null,null,null,null


In [42]:
add = chkpt0.select("add").where("add is not null")
add.printSchema()
a = add.withColumnRenamed("add", "a")
a.select("a.*").show(100, truncate=False)

root
 |-- add: struct (nullable = true)
 |    |-- path: string (nullable = true)
 |    |-- partitionValues: map (nullable = true)
 |    |    |-- key: string
 |    |    |-- value: string (valueContainsNull = true)
 |    |-- size: long (nullable = true)
 |    |-- modificationTime: long (nullable = true)
 |    |-- dataChange: boolean (nullable = true)
 |    |-- stats: string (nullable = true)
 |    |-- tags: map (nullable = true)
 |    |    |-- key: string
 |    |    |-- value: string (valueContainsNull = true)

+-------------------------------------------------------------------+---------------+----+----------------+----------+-----+----+
|path                                                               |partitionValues|size|modificationTime|dataChange|stats|tags|
+-------------------------------------------------------------------+---------------+----+----------------+----------+-----+----+
|part-00000-f05b7216-70cb-45c2-bd80-b2ea8c63397b-c000.snappy.parquet|[]             |1189|16289

![cache-4](images/figure.1-14.png)

```json
stats_parsed:{
    "numRecords": 7,
    "minValues": {
        "addr_state": "IA",
        "count": 3,
        "stream_no": 3
    },
    "maxValues": {
        "addr_state": "TX",
        "count": 9,
        "stream_no": 3
    },
    "nullCount": {
        "addr_state": 0,
        "count": 0,
        "stream_no": 0
    }
}
```

> 스파크는 페타바이트 규모의 데이터를 읽을 때에 모든 데터를 읽지 않고도 stats_parsed 항목의 통계정보를 통해 보다 빠르게 통계정보를 획득할 수 있습니다.


### 4-3. Dealing With Multiple Concurrent Reads and Writes

> 여태까지 우리는 하이레벨 Delta Lake 트랜젝션 로그를 이해했습니다. 이제는 동시성에 대해 알아봅니다. Delta Lake 는 어떻게 다수의 동시성 읽기와 쓰기를 다루는지 알아보겠습니다. 스파크 기반의 데이터 처리를 수행하므로, 하나의 테이블에 대해 다수의 이용자가 사용하게 되며, Delta Lake 는 `낙관적 동시성 제어`를 합니다.

#### What Is Optimistic Concurrency Control?

> '낙관적 동시성 제어'는 다수의 이용자가 하나의 테이블을 다룰 때에 서로 충돌하지 않고 트랜잭션이 잘 완료될 것이라는 가정을 말합니다. 이는 수 페타바이트 규모의 데이터를 다룬다고 가정한다면 이용자들은 서로 다른 부분의 데이터를 다루고 있을 가능성이 더 높을 것입니다. 테이블을 다룰 때에 서로 다른 클라이언트들이 테이블의 서로 다른 부분을 수정하거나, 충돌이 발생하지 않는 액션을 수행하는 한, 그러한 오퍼레이션들은 문제가 되지 않기 때문에, 우리는 '난꽌적으로' 그 작업을 완수할 수 있습니다.

> 반면 클라이언트가 동시에 같은 부분을 수정하는 경우, Delta Lake 는 이를 해결하기 위한 프로토콜을 가지고 있습니다.

<br>

#### Solving conflicts optimistically

##### 가. Ensuring serializability : `상호 배제 (Mutual Exclusion)` 즉, 여러 writers 가 있다고 하더라도 소위 데이터베이스에서의 `serializability` 변경 순서대로 수행되는 속성을 보장합니다. **사건이 동시에 발생하더라도, 마치 순차적으로 발생한 것 처럼 수행**합니다.

![user-1](images/figure.1-15.png)

![user-2](images/figure.1-16.png)

> "User 1"이 `00002.json` 파일을 커밋하려고 할 때에, "User 2"가 이미 점유하고 있다면 "User 1"은 이미 `000002.json`이 존재함을 알고, 상호배제에 따라 "커밋에 실패했군" 이라고 말하고, `00003.json` 파일을 커밋하게 됩니다.

![serializability](images/figure.1-17.png)

<br>




##### 나. Applying Optimistic Concurrency Control in Delta.

> ACID 트랜잭션을 제공하기 위해서 Delta Lake 는 **어떻게 커밋들이 순서지어 져야 하는지(concept of `serializability` in databases) 규정하는**, 그리고 **두개 이상의 커밋이 동시에 수행되어질 때에 어떻게 처리해야 할 지를 결정**하는 `프로토콜`을 가집니다.

> Delta Lake 는 이러한 문제를 `mutual exclusion` 의 규칙에 의해 구현되며, 어떤 충돌에 대해서도 낙관적으로 해결하려고 시도합니다. 이 프로토콜은 Delta Lake 로 하여금, `isolation` 의 ACID 원칙을 제공해주도록 허용합니다. 여기서 `isolation` 이란 테이블에 대한 다수의 동시성 writes 의 결과 상태가 마치 writes 들이 서로 격리되고, 순차적으로 발생한 것처럼 수행되는 것을 보장합니다. 대게 아래의 5단계를 통해 수행됩니다.

* a. 테이블 시작 버전을 기록합니다
* b. 읽기/쓰기를 기록합니다
* c. 커밋을 시도합니다
* d. 만일 누군가가 해당 리소스를 이미 획득(win)하고 있다면, 당신이 읽으려고 하는 것이 변경되었는지를 확인합니다
* e. 다시 반복합니다

> 예를 들어, 2명의 이용자가 동일한 테이블을 읽고, 동시에 데이터를 삽입하려고 한다면 ?

![conflict](images/figure.1-18.png)

* a. Delta Lake는 변경하기 전에 읽은 테이블(버전 0)의 시작 테이블 버전을 기록합니다.
* b. 사용자 1과 2는 모두 동시에 테이블에 일부 데이터를 추가하려고 시도합니다. 여기에서 한 커밋만 다음에 올 수 있고 000001.json으로 기록될 수 있기 때문에 충돌이 발생했습니다.
* c~d. Delta Lake는 상호배제 개념으로 이 충돌을 처리합니다. 즉, 한 명의 사용자만 000001.json을 성공적으로 커밋할 수 있어서, 사용자 1의 커밋은 수락되고 사용자 2의 커밋은 거부됩니다.
* e. Delta Lake는 사용자 2에 대해 오류를 발생시키는 대신 이 충돌을 낙관적으로 처리하는 것을 선호합니다. 테이블에 대한 새로운 커밋이 있는지 확인하고 해당 변경 사항을 반영하도록 테이블을 자동으로 업데이트한 다음 데이터 처리 없이 새로 업데이트된 테이블에서 사용자 2의 커밋을 재시도하여 000002.json을 성공적으로 커밋합니다.

> 대부분의 경우 이러한 조정은 조용하고 원활하며 성공적으로 이루어집니다. 그러나 Delta Lake가 낙관적으로 해결할 수 없는 양립할 수 없는 문제가 있는 경우 (예: 사용자 1이 사용자 2도 삭제한 파일을 삭제한 경우) 유일한 옵션은 오류를 발생시키는 것입니다. 

> 끝으로 Delta Lake 테이블에서 이루어진 모든 트랜잭션은 스토리지에 직접 저장되기 때문에 이 프로세스는 `durability`라는 ACID 속성을 만족하므로 시스템 장애 시에도 지속됩니다.

<br>



##### 다. Multiversion Concurrency Control.

> 이러한 일들이 파일 시스템 내에서 수행될 수 있는 것은 Delta 의 트랜잭션들이 `Multiversion Concurrency Control (MVCC)`을 이용하여 구현되었기 때문입니다. 이는 **관계형 데이터베이스 관리 시스템에서 데이터베이스에 동시에 접근하는 경우에 흔히 사용되는 동시성 제어 방법**입니다. 

> Delta Lake 의 데이터 객체와 로그들은 `immutable` 속성을 가지며, Delta Lake 는 MVCC 를 이용하여 **기 존재하는 데이터를 보호**하고(예를 들어 writes 들 간에 트랜잭션을 보장하는), 더불어 **질의 속도와 쓰기 퍼포먼스를 향상**시킵니다. 이러한 매커니즘을 통해 쓰기 연산은 아래의 3가지 스테이지를 가집니다.

* *Read* : 수정이 필요한 로우들을 식별하기 위해서 최근의 가용한 테이블의 버전을 읽어옵니다

* *Write* : 새로운 데이터 파일들을 쓰게되어 발생한 모든 변경사항들을 스테이지 합니다. 여기서 발생하는 모든 삽입 및 수정사항 들은 새로운 파일들의 형태로 저장됩니다.

* *Validate and commit* : 변경사항들을 커밋하기 전에, 읽어왔던 스냅샷 시점 이후에 동시에 커밋이 이루어진 어떠한 변경 사항들에 대해서도 충돌여부를 확인합니다. 충돌이 없다면 모든 스테이징된 변경 사항은 커밋되어 새로운 버전의 스냅샷으로 생성됩니다. 반면에 충돌이 발생한 경우에, 쓰기 연산은 '동시 변경 예외'를 발생시키고 실패하는데, Parqeut 테이블에 쓰기에 실패했을 때와 같이 손상시키게 됩니다.

<br>

> 테이블의 변경사항이 늘어남에 따라 Delta의 MVCC 알고리즘은 **업데이트 또는 제거 중인 레코드가 포함된 파일을 즉시 교체하지 않고 여러 데이터 복사본을 유지**합니다. MVCC는 테이블 상태의 일관된 보기(consistent view)에 대한 직렬화(serializability) 및 스냅샷 격리(snapshot isolation)를 허용하므로 reader 들은 작업 중에 테이블이 수정된 경우에도 Apache Spark 작업이 시작된 테이블의 일관된 스냅샷 보기를 계속 볼 수 있습니다. writers 들이 동시에 수정 작업을 할 때 트랜잭션 로그를 사용하여 처리할 데이터 파일을 선택적으로 골라, 스냅샷을 효율적으로 쿼리할 수 있습니다. writers 가 테이블 수정 시에는 아래의 2개의 페이즈를 수행합니다.

* a. 새로운 파일들 혹은 갱신된 이미 존재하는 것들의 복제본에 대해 낙관적인 쓰기를 수행합니다
* b. 그런다음 커밋 후, 로그에 새로운 엔트리르 추가함으로써 테이블의 원자적(atomic) 최신 버전을 생성합니다. 이 로그 엔트리에는 테이블에 대한 다른 메타데이터의 변경 사항과 함께 논리적으로 추가 및 제거할 데이터 파일이 기록됩니다

### 4-4. Other Use Cases

#### Time Travel

> 모든 테이블은 많고 작음의 차이는 있더라도, `Delta Lake 트랜잭션 로그의 저장된 커밋들의 집합체`라고 말할 수 있습니다. *트랜잭션 로그는 순차적 명령어 가이드*라고 말할 수 있는데, 최초의 상태에서 현재의 상태에 이르기 까지의 과정을 설명한다고 말할 수 있습니다. 그래서 우리는 테이블의 어던 특정 시점에서부터 시작하더라도 상태를 재구성할 수 있으며, **time travel** 혹은 *data versioning* 이라고 알려진 강력한 능력을 가지게 됩니다. 

> 추가적인 이해는 [Introducing Delta Time Travel for Large Scale Data Lakes](https://databricks.com/blog/2019/02/04/introducing-delta-time-travel-for-large-scale-data-lakes.html) 자료와 [Query an older snapshot of a table (time travel)](https://docs.databricks.com/delta/delta-batch.html?_ga=2.97316378.466340956.1612291545-1203283123.1598416965#query-an-older-snapshot-of-a-table-time-travel) 을 통해 학습할 수 있습니다.


#### Data Lineage and Debugging

> 모든 변화를 유지하는 트랜잭션 로그의 특징 덕분에 Delta Lake 트랜잭션 로그를 통해 `governance, 감사(audit) 그리고 규정준수(compliance purpose)를 위한 검증 가능한 data lineage`를 제공받을 수 있습니다. 즉, 의도하지 않은 변경의 원인을 추적하거나, 파이프라인의 버그를 찾을 때에 유용할 것입니다. `DESCRIBE HISTORY` 명령을 이용하여 변경된 메타데이터 정보를 통해 확인할 수 있습니다

> 추가적인 명령어는 [Table utility commands](https://docs.delta.io/latest/delta-utility.html#history) 페이지에서 찾아볼 수 있습니다


### 4-5. Diving further into the transaction log

> 이번 섹션에서는 Delta Lake 트랜잭션 로그가 어떻게 동작하는지에 대해 학습하였습니다.

* 트랜잭션 로그가 무엇인지, 어떻게 구성되어 있는지, 커밋이 디스크에 파일로 저장되는 방식.
* Delta Lake가 원자성 원칙을 구현할 수 있도록 트랜잭션 로그가 단일 정보 소스 역할을 하는 방법.
* Delta Lake가 각 테이블의 상태를 계산하는 방법 - 트랜잭션 로그를 사용하여 가장 최근의 체크포인트를 따라잡는 방법을 포함합니다.
* 낙관적 동시성 제어를 사용하여 테이블이 변경되더라도 여러 동시 읽기 및 쓰기를 허용합니다.
* Delta Lake가 커밋이 올바르게 직렬화되도록 하기 위해 상호 배제를 사용하는 방법과 충돌 발생 시 자동으로 재시도하는 방법.

> 추가로 참고할 만한 세션과 영상입니다 

* Unpacking the Transaction Log
  - [Unpacking the Transaction Log I](https://databricks.com/discover/diving-into-delta-lake-talks/unpacking-transaction-log)
  - [Unpacking the Transaction Log II](https://databricks.com/session_eu20/diving-into-delta-lake-unpacking-the-transaction-log)


[맨 위로](#CHAPTER-1.-Basic-Operations-on-Delta-Lakes)

## 5. Table Utilities

### 5-1. Review table history

> 


In [43]:
df = spark.range(1, 10)
df.write.format("delta").mode("overwrite").saveAsTable("foo")
spark.sql("show tables")

database,tableName,isTemporary
default,foo,false
default,hello,false
default,say,false
default,say_hello,false


In [44]:
spark.sql("describe foo")

col_name,data_type,comment
id,bigint,
,,
# Partitioning,,
Not partitioned,,


In [46]:
df.write.format("delta").mode("overwrite").save("tmp/foo")
spark.sql(f"describe history delta.`{work_dir}/tmp/foo`")

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata
0,2021-08-14 23:38:46.66,null,null,WRITE,"[mode -> Overwrite, partitionBy -> []]",null,null,null,null,null,false,"[numFiles -> 3, numOutputBytes -> 1426, numOutputRows -> 9]",null


In [47]:
from delta.tables import *

deltaTable = DeltaTable.forPath(spark, f"{work_dir}/tmp/foo")
fullHistroyDF = deltaTable.history()
last5OperationsDF = deltaTable.history(5)

In [48]:
display(last5OperationsDF)

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata
0,2021-08-14 23:38:46.66,null,null,WRITE,"[mode -> Overwrite, partitionBy -> []]",null,null,null,null,null,false,"[numFiles -> 3, numOutputBytes -> 1426, numOutputRows -> 9]",null


### 5-2. Vacuum History

> 더 이상 필요하지 않은 과거 이력에 대해서 `VACUUM` 명령을 통해서 삭제할 수 있습니다. 즉, 갱신 혹은 삭제를 통해 더 이상 참조하지 않는 파일들이 존재할 것이며, 이에 대한 정리작업이 필요할 수 있습니다. 

> Delta Table 에 의해 더 이상 참조되지 않으며, Rentention threshold 값 보다 오래된 파일들에 대해서 `VACUUM` 명령어를 통해 삭제할 수 있습니다. 주의할 사항은 VACUUM 명령어는 자동으로 트리거링 되지 않으며, 트리거링 되었을 때에 기본 Retention 은 7일입니다. 

> 예를 들어, 7일 이상 지난 레퍼런스된 파일에 대해서는 더 이상 삭제되지 않는다는 의미입니다.

In [49]:
spark.sql("vacuum foo dry run")

path


In [50]:
# spark.sql("vacuum delta.'/home/jovyan/work/tmp/foo'")
# spark.sql("vacuum foo")
# spark.sql("vacuum foo retain 100 hours")
spark.sql("VACUUM foo RETAIN 168 HOURS")

path
file:/home/jovyan/work/lgde-spark-delta/spark-warehouse/foo


In [51]:
spark.sql("describe history foo")

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata
0,2021-08-14 23:38:16.531,null,null,CREATE OR REPLACE TABLE AS SELECT,"[isManaged -> true, description ->, partitionBy -> [], properties -> {}]",null,null,null,null,null,false,"[numFiles -> 3, numOutputBytes -> 1426, numOutputRows -> 9]",null


In [52]:
deltaTable = DeltaTable.forPath(spark, f"{work_dir}/tmp/foo")
deltaTable = DeltaTable.forName(spark, "foo")

# 아래와 같이 Delta Table 은 데이터프레임과 달라 저장할 수 없기 때문에 toDF 메소드로 변환이 필요하지만, 스키마가 달라 오류가 발생한다
# deltaTable.toDF().write.format("delta").mode("overwrite").save("/home/jovyan/work/tmp/copy_of_say_hello")

# 스키마가 다른 경우 '.option("overwriteSchema", "true")' 을 통해 overwrite 할 수 있다
deltaTable.toDF().write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(f"{work_dir}/tmp/copy_of_say_hello")

In [53]:
copy_of_say_hello = spark.read.format("delta").load("tmp/copy_of_say_hello")
copy_of_say_hello.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("copy_of_say_hello")
spark.sql("describe history copy_of_say_hello")

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata
0,2021-08-14 23:41:19.4,null,null,CREATE OR REPLACE TABLE AS SELECT,"[isManaged -> true, description ->, partitionBy -> [], properties -> {""overwriteSchema"":""true""}]",null,null,null,null,null,false,"[numFiles -> 3, numOutputBytes -> 1426, numOutputRows -> 9]",null


In [55]:
# 7일 이하로 vacuum 시도 시에 아래와 같은 오류 메시지를 뿌리고, duration check 를 false 로 해야 수행된다고 한다
spark.sql("set spark.databricks.delta.retentionDurationCheck.enabled = true")
spark.sql(f"vacuum delta.`{work_dir}/tmp/foo` retain 10 hours")

IllegalArgumentException: requirement failed: Are you sure you would like to vacuum files with such a low retention period? If you have
writers that are currently writing to this table, there is a risk that you may corrupt the
state of your Delta table.

If you are certain that there are no operations being performed on this table, such as
insert/upsert/delete/optimize, then you may turn off this check by setting:
spark.databricks.delta.retentionDurationCheck.enabled = false

If you are not sure, please use a value not less than "168 hours".
       

In [56]:
# 시간 단위가 아니라 분 단위로도 설정이 가능하고, 기존에 3회 반복적으로 write 한 이력을 확인할 수 있었다
# 6분 전이 아닌 과거의 파케이 파일들은 삭제가 되었으나, `_delta_log` 에는 이력이 존재하긴 한다
spark.sql("set spark.databricks.delta.retentionDurationCheck.enabled = false")
spark.sql("set spark.databricks.delta.vacuum.parallelDelete.enabled = true")
spark.sql(f"vacuum delta.`{work_dir}/tmp/foo` retain 0.1 hours")

path
file:/home/jovyan/work/lgde-spark-delta/tmp/foo


In [57]:
spark.sql(f"describe history delta.`{work_dir}/tmp/foo`")

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata
0,2021-08-14 23:38:46.66,null,null,WRITE,"[mode -> Overwrite, partitionBy -> []]",null,null,null,null,null,false,"[numFiles -> 3, numOutputBytes -> 1426, numOutputRows -> 9]",null


In [58]:
# 가장 마지막 상태인 파일들로만 구성된 파케이 파일로 조회해도 결과는 동일하다
spark.sql(f"select * from delta.`{work_dir}/tmp/foo`")

id
4
5
6
7
8
9
1
2
3


In [59]:
# 과거 특정시점으로 이동이 가능한지 확인해보았다
spark.sql(f"describe history delta.`{work_dir}/tmp/foo`")

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata
0,2021-08-14 23:38:46.66,null,null,WRITE,"[mode -> Overwrite, partitionBy -> []]",null,null,null,null,null,false,"[numFiles -> 3, numOutputBytes -> 1426, numOutputRows -> 9]",null


In [60]:
# 이미 삭제된 FileNotFoundException 파일이라고 나오면서 오류가 발생
v0 = spark.read.format("delta").option("versionAsOf", 0).load(f"{work_dir}/tmp/foo")
display(v0)

id
4
5
6
7
8
9
1
2
3


In [61]:
# 현재 존재하는 버전을 가져올 때에는 정상적으로 조회가 된다
v2 = spark.read.format("delta").option("versionAsOf", 2).load(f"{work_dir}/tmp/foo")
display(v2)

AnalysisException: Cannot time travel Delta table to version 2. Available versions: [0, 0].;

#### Configure Log and Data History
> [spark.databricks.delta Configuration](https://books.japila.pl/delta-lake-internals/DeltaSQLConf/) 페이지를 참고합니다. 

##### Log history. : Delta Transaction Log 가 지정된 리텐션 기간을 넘어서는 커밋이 발생하는 경우 과거의 트랜잭션 로그는 삭제됩니다.
* `spark.databricks.delta.logRetentionDuration` : default = 30 days
* 이는 Delta Log 가 무한정 증가하는 것을 막기 위해 고안된 사항이며 기본값은 30일입니다
  - 물리적인 파일과는 다르게 **트랜잭션 로그는 자동적으로 삭제** 됩니다.

##### Data history. : 
* `spark.databricks.delta.deletedFileRetentionDuration` : defatul = 7 days
* 필요 없게된 데이터 파일을 삭제하기 위한 기능이며 기본값은 7일입니다
  - 트랜잭션 로그와는 다르게 **물리적인 데이터 파일은 삭제되지 않으**므로 `VACUUM` 명령으로 스케줄링 되어야만 합니다

##### Parallel deletion of files during vacuum.
* `spark.databricks.delta.vacuum.parallelDelete.enabled` : default = false
* `VACUUM` 명령 수행 시에 병렬로 데이터 파일을 삭제할 수 있으며 기본값은 false 입니다
  - 파일 삭제 시에 셔플파티션 수가 많은 경우 병렬로 설정이 가능하며 세션 설정에서 변경할 수 있습니다

##### Preventing very short retetion period vacuum.
* `spark.databricks.delta.retentionDurationCheck.enabled` : default = true
* 리텐션 인터벌을 7일 이하로 지정하는 것을 권장하지 않으며 기본 설정은 true 입니다. 
  - **대상 테이블에 concurrent readers 혹은 writers 들이 오래된 스냅샷 혹은 커밋되지 않은 파일에 존재**할 수 있기 때문입니다.
* 반드시 가장 길다고 판단되는 동시성 트랜잭션 보다 길게 인터벌을 지정해야 의도하지 않은 데이터파일 삭제를 막을 수 있습니다.

<br>

### 5-3. Retrieve Delta table details

### 5-4. Generate a manifest file

### 5-5. Convert a Parquet table to a Delta table

### 5-6. Convert a Delta table to a Parquet table

### 5-7. Restore a table version

## 6. Summary


[맨 위로](#CHAPTER-1.-Basic-Operations-on-Delta-Lakes)

### Update Table w/ Overwrite

In [12]:
data = spark.range(5, 10)
data.write.format("delta").mode("overwrite").save("tmp/delta-table")

### Conditional update without overwrite

In [13]:
from delta.tables import *
from pyspark.sql.functions import *

deltaTable = DeltaTable.forPath(spark, "tmp/delta-table")

# Update every even value by adding 100 to it
deltaTable.update(
  condition = expr("id % 2 == 0"),
  set = { "id": expr("id + 100") })

# Delete every even value
deltaTable.delete(condition = expr("id % 2 == 0"))

# Upsert (merge) new data
newData = spark.range(0, 20)

deltaTable.alias("oldData") \
  .merge(
    newData.alias("newData"),
    "oldData.id = newData.id") \
  .whenMatchedUpdate(set = { "id": col("newData.id") }) \
  .whenNotMatchedInsert(values = { "id": col("newData.id") }) \
  .execute()

deltaTable.toDF().show()

+---+
| id|
+---+
|  5|
|  1|
|  3|
|  7|
|  0|
|  4|
|  9|
| 18|
| 19|
| 11|
|  2|
| 15|
| 16|
| 13|
| 14|
| 10|
| 12|
|  6|
|  8|
| 17|
+---+



In [14]:
deltaTable.toDF().sort("id").show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
| 15|
| 16|
| 17|
| 18|
| 19|
+---+



### Read older versions of data using time travel

In [15]:
df = spark.read.format("delta").option("versionAsOf", 0).load("tmp/delta-table")
df.show()

+---+
| id|
+---+
|  6|
|  7|
|  8|
|  9|
|  5|
+---+



### Write a stream of data to a table


In [16]:
streamingDf = spark.readStream.format("rate").load()
stream = streamingDf.selectExpr("value as id").writeStream.format("delta").option("checkpointLocation", "tmp/checkpoint").start("tmp/delta-table")

### Read a stream of changes from a table

In [17]:
stream2 = spark.readStream.format("delta").load("/tmp/delta-table").writeStream.format("console").start()

AnalysisException: Table schema is not set.  Write data into it or use CREATE TABLE to set the schema.

### Handling Delta using Spark SQL Magic Function

In [ ]:
df = spark.sql("select 1 as col1, 1L as col2")
df.printSchema()
df2 = df.withColumn("col3", expr("case when col2 is not null then 10 else col2 end"))
df2.printSchema()
df2.show()

In [ ]:
print("a,b".find(","))

In [ ]:
df = spark.sql("select 'a,b' as col1")
def splitColumnA(col, sep):
    return split(col, sep)[0]
def splitColumnB(col, sep):
    return split(col, sep)[1]
df2 = df.withColumn("col2", splitColumnA(expr("col1"), ",")).withColumn("col3", splitColumnB(expr("col1"), ","))
df3 = df2.na.fill({"col3": "default_value"})
df3.show()

In [ ]:
%load_ext sparksql_magic

In [ ]:
%%sparksql

CREATE TABLE events (
  date DATE,
  eventId STRING,
  eventType STRING,
  data STRING)
USING DELTA

In [ ]:
%%sparksql
CREATE TABLE foo
USING DELTA
LOCATION 'tmp/delta-table'

In [ ]:
%%sparksql
select * from foo order by id asc

### 참고 사이트
* [Data Engineer Training](https://github.com/psyoblade/data-engineer-training)
* [SparkSQL Magic](https://github.com/cryeo/sparksql-magic)